In [90]:
import os
from glob import glob
import pandas as pd
from tqdm.auto import tqdm

# explore folder

In [14]:
# 프로젝트 데이터 폴더
file_dir = "D:visdrone/"
filelist_file_dir = glob(file_dir+"*")
filelist_file_dir

['D:visdrone\\raw_data_zip',
 'D:visdrone\\VisDrone2019-MOT-test-challenge',
 'D:visdrone\\VisDrone2019-MOT-test-dev',
 'D:visdrone\\VisDrone2019-MOT-train',
 'D:visdrone\\VisDrone2019-MOT-val']

In [24]:
# every file_folder
[glob(f+"/*") for f in filelist_file_dir]

[['D:visdrone\\raw_data_zip\\VisDrone2019-MOT-test-challenge.zip',
  'D:visdrone\\raw_data_zip\\VisDrone2019-MOT-test-dev.zip',
  'D:visdrone\\raw_data_zip\\VisDrone2019-MOT-train.zip',
  'D:visdrone\\raw_data_zip\\VisDrone2019-MOT-val.zip'],
 ['D:visdrone\\VisDrone2019-MOT-test-challenge\\sequences'],
 ['D:visdrone\\VisDrone2019-MOT-test-dev\\annotations',
  'D:visdrone\\VisDrone2019-MOT-test-dev\\sequences'],
 ['D:visdrone\\VisDrone2019-MOT-train\\annotations',
  'D:visdrone\\VisDrone2019-MOT-train\\sequences'],
 ['D:visdrone\\VisDrone2019-MOT-val\\annotations',
  'D:visdrone\\VisDrone2019-MOT-val\\sequences']]

In [31]:
val_dir = filelist_file_dir[4]
# val_annotations
val_anno = glob(val_dir + "/annotations/*")
val_anno

['D:visdrone\\VisDrone2019-MOT-val/annotations\\uav0000086_00000_v.txt',
 'D:visdrone\\VisDrone2019-MOT-val/annotations\\uav0000117_02622_v.txt',
 'D:visdrone\\VisDrone2019-MOT-val/annotations\\uav0000137_00458_v.txt',
 'D:visdrone\\VisDrone2019-MOT-val/annotations\\uav0000182_00000_v.txt',
 'D:visdrone\\VisDrone2019-MOT-val/annotations\\uav0000268_05773_v.txt',
 'D:visdrone\\VisDrone2019-MOT-val/annotations\\uav0000305_00000_v.txt',
 'D:visdrone\\VisDrone2019-MOT-val/annotations\\uav0000339_00001_v.txt']

In [32]:
# val_annotations
val_seq = glob(val_dir + "/sequences/*")
val_seq

['D:visdrone\\VisDrone2019-MOT-val/sequences\\uav0000086_00000_v',
 'D:visdrone\\VisDrone2019-MOT-val/sequences\\uav0000117_02622_v',
 'D:visdrone\\VisDrone2019-MOT-val/sequences\\uav0000137_00458_v',
 'D:visdrone\\VisDrone2019-MOT-val/sequences\\uav0000182_00000_v',
 'D:visdrone\\VisDrone2019-MOT-val/sequences\\uav0000268_05773_v',
 'D:visdrone\\VisDrone2019-MOT-val/sequences\\uav0000305_00000_v',
 'D:visdrone\\VisDrone2019-MOT-val/sequences\\uav0000339_00001_v']

In [34]:
# val_seq 1 folder sample
glob(val_seq[0]+"/*")

['D:visdrone\\VisDrone2019-MOT-val/sequences\\uav0000086_00000_v\\0000001.jpg',
 'D:visdrone\\VisDrone2019-MOT-val/sequences\\uav0000086_00000_v\\0000002.jpg',
 'D:visdrone\\VisDrone2019-MOT-val/sequences\\uav0000086_00000_v\\0000003.jpg',
 'D:visdrone\\VisDrone2019-MOT-val/sequences\\uav0000086_00000_v\\0000004.jpg',
 'D:visdrone\\VisDrone2019-MOT-val/sequences\\uav0000086_00000_v\\0000005.jpg',
 'D:visdrone\\VisDrone2019-MOT-val/sequences\\uav0000086_00000_v\\0000006.jpg',
 'D:visdrone\\VisDrone2019-MOT-val/sequences\\uav0000086_00000_v\\0000007.jpg',
 'D:visdrone\\VisDrone2019-MOT-val/sequences\\uav0000086_00000_v\\0000008.jpg',
 'D:visdrone\\VisDrone2019-MOT-val/sequences\\uav0000086_00000_v\\0000009.jpg',
 'D:visdrone\\VisDrone2019-MOT-val/sequences\\uav0000086_00000_v\\0000010.jpg',
 'D:visdrone\\VisDrone2019-MOT-val/sequences\\uav0000086_00000_v\\0000011.jpg',
 'D:visdrone\\VisDrone2019-MOT-val/sequences\\uav0000086_00000_v\\0000012.jpg',
 'D:visdrone\\VisDrone2019-MOT-val/seque

In [35]:
# images counts per val_seq folder
for seq in val_seq:
    print(seq, "\t folder length \t:", len(glob(seq+"/*")))

D:visdrone\VisDrone2019-MOT-val/sequences\uav0000086_00000_v 	 folder length 	: 464
D:visdrone\VisDrone2019-MOT-val/sequences\uav0000117_02622_v 	 folder length 	: 349
D:visdrone\VisDrone2019-MOT-val/sequences\uav0000137_00458_v 	 folder length 	: 233
D:visdrone\VisDrone2019-MOT-val/sequences\uav0000182_00000_v 	 folder length 	: 363
D:visdrone\VisDrone2019-MOT-val/sequences\uav0000268_05773_v 	 folder length 	: 978
D:visdrone\VisDrone2019-MOT-val/sequences\uav0000305_00000_v 	 folder length 	: 184
D:visdrone\VisDrone2019-MOT-val/sequences\uav0000339_00001_v 	 folder length 	: 275


# explore annotation

# make dataframe

In [36]:
os.val_anno[0]

'D:visdrone\\VisDrone2019-MOT-val/annotations\\uav0000086_00000_v.txt'

In [62]:
with open(val_anno[0],"r") as f:
    line = f.readline()
    line = line.split()
    for i in line:
        print(i)

102,0,38,666,71,88,1,1,1,0


0. "frame_index" : 비디오 프레임의 프레임 인덱스

1. "target_id"	 : DETECTION 결과 파일에서 대상의 ID는 상수 -1로 설정되어야 합니다. GROUNDTRUTH 파일에서 대상의 ID는 다른 프레임에서 경계 상자의 시간적 대응 관계를 제공하는 데 사용됩니다.

2. bbox_x :	bbox의 좌상단점의 x 좌표

3. bbox_y :	bbox의 좌상단점의 y 좌표

4. bbox_width : bbox 의 너비

5. bbox_height : bbox 의 높이

6. score : bbox의 신뢰도, GROUNDTRUTH 에서는 1, 0 으로 표시되고 1이 bbox가 평가 되어야 함을 나타냅니다.
    
7. object_category : 
    * ignored regions (0)
    * pedestrian (1)
    * people (2)
    * bicycle (3)
    * car (4)
    * van (5)
    * truck (6)
    * tricycle (7)
    * awning-tricycle (8)
    * bus (9)
    * motor (10)
    * others (11))
    
8. truncation : 객체가 이미지 프레임에 의해 잘려진 정도를 나타내며 0, 1 의 값을 가집니다. 이때 0은 잘림이 없고 1이 부분적으로 잘렸음(1~50%)을 나타냅니다.
    
9. occlusion : 객체가 가려진 정도를 나타내며 0, 1, 2 의 값을 가집니다. 이때 0은 0%, 1은 (1%~50%), 2는 (50%~100%) 를 나타냅니다.

# make dataframe

In [88]:
df = pd.DataFrame(columns=["video_name","frame_index","target_id" ,"xywh","score","class","truncation","occlusion"]); df

,video_name,frame_index,target_id,xywh,score,class,truncation,occlusion


In [96]:
df = pd.DataFrame(columns=["video_name","frame_index","target_id" ,"xywh","score","class","truncation","occlusion"])

idx = 0
for file in tqdm(val_anno):
    base, name = os.path.split(file)
    
    name, ex = os.path.splitext(name)
    base, _ = os.path.split(base)
    
    video_name = base+"/sequences/"+name+"/"

    with open(file,"r") as f:
        while True:
            line = f.readline().strip()
            if not line:
                break
            line = line.split(",")
            df.loc[idx]=[
                video_name, # video name : str
                line[0].zfill(7), # frame number : str
                int(line[1]), # target_id : int
                list(map(int, line[2:6])), # xywh : list[int]
                int(line[6]), # score : int
                int(line[7]), # class : int
                int(line[8]), # "truncation" : int
                int(line[9]), # "occlusion" : int
            ]
            idx +=1

  0%|          | 0/7 [00:00<?, ?it/s]

In [99]:
df.to_csv("dataframe.csv", index=False)